# Ch14： Recurrent Neural Networks

## Basic RNNs in TensorFlow

![Alt](./14_2.png)

In [2]:
import tensorflow as tf
n_inputs=3
n_neurons=5

X0=tf.placeholder(tf.float32,[None,n_inputs])
X1=tf.placeholder(tf.float32,[None,n_inputs])

Wx=tf.Variable(tf.random_normal(shape=[n_inputs,n_neurons],dtype=tf.float32))
Wy=tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b=tf.Variable(tf.zeros([1,n_neurons],dtype=tf.float32))

Y0=tf.tanh(tf.matmul(X0,Wx)+b)
Y1=tf.tanh(tf.matmul(Y0,Wy)+tf.matmul(X1,Wx)+b)

init=tf.global_variables_initializer()

In [3]:
import numpy as np
X0_batch=np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch=np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

with tf.Session() as sess:
    init.run()
    Y0_val,Y1_val=sess.run([Y0,Y1],feed_dict={X0:X0_batch,X1:X1_batch})

In [4]:
print(Y0_val)

[[-0.17640039 -0.57224655 -0.99719042 -0.16053186  0.99741298]
 [ 0.38764769 -0.99663621 -1.          0.99994576  0.99994975]
 [ 0.76004553 -0.9999792  -1.          1.          0.99999905]
 [-0.96280271 -0.99324793 -1.          1.         -0.99999988]]


In [5]:
print(Y1_val)

[[ 0.88383186 -0.99999994 -1.          1.          0.66440773]
 [ 0.28166419 -0.939812    0.63064516  0.99659473 -0.9927938 ]
 [ 0.94650102 -0.99997854 -1.          1.         -0.99412483]
 [-0.91097897 -0.99999619 -0.9999994   0.99999797  0.28794128]]


## Static Unrolling Through Time

In [6]:
tf.reset_default_graph()
X0=tf.placeholder(tf.float32,[None,n_inputs])
X1=tf.placeholder(tf.float32,[None,n_inputs])

basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs,states=tf.contrib.rnn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32)

Y0,Y1=output_seqs

In [7]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    Y0_val_s,Y1_val_s=sess.run([Y0,Y1],feed_dict={X0:X0_batch,X1:X1_batch})

In [8]:
print(Y0_val_s)

[[-0.21036781  0.62269402 -0.73273349 -0.57639426 -0.66842997]
 [-0.92657721  0.97495639 -0.99965382 -0.76476115 -0.75942796]
 [-0.99555725  0.99861777 -0.99999964 -0.87595707 -0.82802624]
 [-0.99854875  0.99938279 -0.99822694  0.881194    0.99997175]]


In [46]:
tf.reset_default_graph()
n_steps=2
X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
X_seqs=tf.unstack(tf.transpose(X,perm=[1,0,2]))
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs,states=tf.contrib.rnn.static_rnn(basic_cell,X_seqs,dtype=tf.float32)
outputs=tf.transpose(tf.stack(output_seqs),perm=[1,0,2])

In [47]:
X_batch=np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]],
])

In [48]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val=outputs.eval(feed_dict={X:X_batch})

In [50]:
print(outputs_val)

[[[ 0.859366    0.58419371  0.83167094  0.773076   -0.15731491]
  [ 0.98212183  0.99998742  0.44587341 -0.98545146 -0.99919242]]

 [[ 0.97351664  0.99526483  0.97424823  0.60056984 -0.9511109 ]
  [ 0.91413325 -0.03092584 -0.96910113 -0.00953994  0.89254838]]

 [[ 0.99524945  0.99995703  0.99630409  0.34534392 -0.99827683]
  [ 0.95794982  0.99945277 -0.87225997 -0.9305886  -0.95641518]]

 [[-0.999798    0.99999243 -0.99960452 -0.99842173 -0.96516532]
  [-0.4336051   0.98613524 -0.35819459  0.08651786 -0.94640535]]]


### Dynmaic Unrolling Through Time

In [53]:
tf.reset_default_graph()
X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs,states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)

### Handling Variable Length Input Sequences

In [57]:
tf.reset_default_graph()
seq_length=tf.placeholder(tf.int32,[None])
n_setps=2
X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
X_seqs=tf.unstack(tf.transpose(X,perm=[1,0,2]))
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs,states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32,sequence_length=seq_length)


In [58]:
X_batch=np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
])
seq_length_batch=np.array([2,1,2,2])

In [59]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val,states_val=sess.run([outputs,states],feed_dict={X:X_batch,seq_length:seq_length_batch})
    

In [60]:
print(outputs_val)

[[[ 0.12516126  0.63551056  0.59586692  0.53054917  0.23055492]
  [-0.99999934 -0.77043694  0.99999851  0.9973703  -0.98947817]]

 [[-0.98314834  0.673204    0.99586034  0.96472472 -0.42871615]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.99988776  0.70769632  0.99996614  0.99790025 -0.81822425]
  [-0.99972206 -0.54172385  0.99976474  0.97281468 -0.9927848 ]]

 [[-0.99995035 -0.91577137  0.99969298 -0.0180788  -0.96920443]
  [-0.92923605  0.20204246  0.89805973  0.68673497 -0.93606234]]]


In [61]:
print(states_val)

[[-0.99999934 -0.77043694  0.99999851  0.9973703  -0.98947817]
 [-0.98314834  0.673204    0.99586034  0.96472472 -0.42871615]
 [-0.99972206 -0.54172385  0.99976474  0.97281468 -0.9927848 ]
 [-0.92923605  0.20204246  0.89805973  0.68673497 -0.93606234]]


### Handling Variable-Length Output Sequence
Nothing to code

## Training RNNs

In [64]:
tf.reset_default_graph()
from tensorflow.contrib.layers import fully_connected

n_steps=28
n_inputs=28
n_neurons=150
n_outputs=10

learning_rate=0.001

X=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y=tf.placeholder(tf.int32,[None])

basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs,states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)

logits=fully_connected(states,n_outputs,activation_fn=None)
xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
loss=tf.reduce_mean(xentropy)

optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(loss)
correct=tf.nn.in_top_k(logits,y,1)
accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

init=tf.global_variables_initializer()

In [66]:
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets("/tmp/data")
X_test=mnist.test.images.reshape(-1,n_steps,n_inputs)
y_test=mnist.test.labels

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [67]:
n_epochs=100
batch_size=150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            X_batch=X_batch.reshape(-1,n_steps,n_inputs)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:X_test,y:y_test})
        print(epoch,"Training accuracy:",acc_train,"Test accuracy:",acc_test)

0 Training accuracy: 0.92 Test accuracy: 0.9184
1 Training accuracy: 0.966667 Test accuracy: 0.9492
2 Training accuracy: 0.94 Test accuracy: 0.9496
3 Training accuracy: 0.966667 Test accuracy: 0.961
4 Training accuracy: 0.973333 Test accuracy: 0.9709
5 Training accuracy: 0.973333 Test accuracy: 0.9712
6 Training accuracy: 0.96 Test accuracy: 0.9639
7 Training accuracy: 0.993333 Test accuracy: 0.9709
8 Training accuracy: 0.966667 Test accuracy: 0.9656
9 Training accuracy: 0.993333 Test accuracy: 0.9683
10 Training accuracy: 0.98 Test accuracy: 0.967
11 Training accuracy: 0.966667 Test accuracy: 0.9771
12 Training accuracy: 1.0 Test accuracy: 0.9737
13 Training accuracy: 0.973333 Test accuracy: 0.9721
14 Training accuracy: 0.966667 Test accuracy: 0.9738
15 Training accuracy: 0.98 Test accuracy: 0.9726
16 Training accuracy: 0.986667 Test accuracy: 0.9761
17 Training accuracy: 0.986667 Test accuracy: 0.9753
18 Training accuracy: 0.98 Test accuracy: 0.9719
19 Training accuracy: 1.0 Test acc